In [214]:
!pip install tensorflow

In [215]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [216]:
# Caminhos para as pastas de dados
train_data_dir = './data'
batch_size = 32 # ***
image_size = (150, 150)
num_epochs = 50
num_classes = 2

In [217]:
# Crie geradores de dados de treinamento
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255, # Normalização das imagens
    validation_split=0.25,  # Define a divisão de validação
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)    

In [218]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

Found 21 images belonging to 2 classes.


In [219]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

Found 6 images belonging to 2 classes.


In [220]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(image_size[0], image_size[1], 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))  # Camada de saída com 1 unidade para classificação binária

In [221]:
# Compile o modelo
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [222]:
# Treine o modelo
model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=validation_generator)

# Salve o modelo treinado
model.save('modeloClassImagens.h5')

print('Modelo salvo com sucesso!')

Epoch 1/50
1/1 [==============================] - 1s 958ms/step - loss: 0.6725 - accuracy: 0.5714 - val_loss: 1.7303 - val_accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 270ms/step - loss: 2.1646 - accuracy: 0.5238 - val_loss: 5.0638 - val_accuracy: 0.5000
Epoch 3/50
1/1 [==============================] - 0s 267ms/step - loss: 4.6109 - accuracy: 0.5238 - val_loss: 3.9022 - val_accuracy: 0.5000
Epoch 4/50
1/1 [==============================] - 0s 265ms/step - loss: 3.8240 - accuracy: 0.5238 - val_loss: 1.5235 - val_accuracy: 0.5000
Epoch 5/50
1/1 [==============================] - 0s 265ms/step - loss: 1.6076 - accuracy: 0.5714 - val_loss: 0.9237 - val_accuracy: 0.5000
Epoch 6/50
1/1 [==============================] - 0s 263ms/step - loss: 1.1625 - accuracy: 0.6667 - val_loss: 1.1219 - val_accuracy: 0.5000
Epoch 7/50
1/1 [==============================] - 0s 263ms/step - loss: 1.2260 - accuracy: 0.4762 - val_loss: 0.8798 - val_accuracy: 0.5000
Epoch 8/50
1/1 [====